In [23]:
import psycopg2


In [24]:
try:
    conn=psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
except psycopg2.Error as e:
    print("Error: could not connect to Postgres database")
    print(e)

In [25]:
try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("could not get a cursor to the Database")
    print(e)

In [26]:
#set auto commit
conn.set_session(autocommit=True)

### In this section: we create fact and dimension tables with Schema Table
#### a fact table and 3 other dimension tables 


In [27]:
### Create a fact table: customer_transactions with columns: Customer_Id, Store_Id, Spent
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer_transactions(Customer_Id int, Store_Id int, Spent float)")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
### Insert Data
try:
    cur.execute("INSERT INTO customer_transactions(Customer_Id, Store_Id, Spent) \
                VALUES (%s,%s,%s)", \
               (1,1,20.50))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)

In [28]:
### Insert Data
try:
    cur.execute("INSERT INTO customer_transactions(Customer_Id, Store_Id, Spent) \
                VALUES (%s,%s,%s)", \
               (2,1,35.21))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)

In [29]:
### CREAT 3 dimension tables
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customers(Customer_Id int, Name text,Rewards boolean )")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
try:
    cur.execute("CREATE TABLE IF NOT EXISTS items_purchased(Customer_Id int, Item_num int, Item_Name text)")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
try:
    cur.execute("CREATE TABLE IF NOT EXISTS store(Store_Id int, State text)")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [30]:
### Insert data
try:
    cur.execute("INSERT INTO customers(Customer_Id, Name,Rewards) \
                VALUES (%s,%s,%s)", \
               (1,"Amanda", True))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)
try:
    cur.execute("INSERT INTO customers(Customer_Id, Name,Rewards) \
                VALUES (%s,%s,%s)", \
               (2,"Toby",False))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)
try:
    cur.execute("INSERT INTO items_purchased(Customer_Id, Item_num, Item_Name) \
                VALUES (%s,%s,%s)", \
               (1,1, "Rubber Soul"))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)
try:
    cur.execute("INSERT INTO items_purchased(Customer_Id, Item_num, Item_Name) \
                VALUES (%s,%s,%s)", \
               (2,3, "Let It Be"))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)
try:
    cur.execute("INSERT INTO store(Store_Id, State) \
                VALUES (%s,%s)", \
               (1, "CA"))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)
try:
    cur.execute("INSERT INTO store(Store_Id, State) \
                VALUES (%s,%s)", \
               (2, "WA"))
except psycog2.Error as e:
    print("Error: insert row")
    print(e)

### Query 1: Find all the customer spending more than 30 dollars, where store they bought, location of store, what they bought, if any rewards!!!

In [31]:
### cutomers.Name, store.Store_Id,store.State,items_purchased.Item_name,customers.Rewards
try:
    cur.execute("SELECT customers.Name, store.Store_Id,store.State,items_purchased.Item_name,customers.Rewards \
                FROM ((customers JOIN customer_transactions ON customers.Customer_Id = customer_transactions.Customer_Id) JOIN \
                items_purchased ON items_purchased.Customer_Id = customer_transactions.Customer_Id) JOIN \
                store ON store.Store_Id = customer_transactions.Store_Id \
                WHERE customer_transactions.Spent >30;")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)
row=cur.fetchone()
while row:
    print(row)
    row=cur.fetchone()

('Toby', 1, 'CA', 'Let It Be', False)


### Query 2: How much did Customer 2 spend?

In [32]:
### customer_id, sum(spent) where customer id=2
try:
    cur.execute("SELECT Customer_Id, SUM(Spent) FROM customer_transactions WHERE Customer_Id=2 GROUP BY Customer_Id;")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)

In [33]:
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(2, 35.21)


In [34]:
### Drop tables
try:
    cur.execute("DROP TABLE customer_transactions, customers, items_purchased, store;")
except psycopg2.Error as e:
    print("Error: Dropping tables")
    print(e)

In [35]:
### Close cursor and connection
cur.close()
conn.close()